## Understanding how to generate reponses using Ollama LLMs

#### 1. Intuition: The "Open-Book Exam"  
Imagine you are a brilliant student (the LLM), but you have amnesia.  
You remember everything up to 2023, but nothing after that.  

- **Without RAG:**  
  If I ask you about news from today, you might panic and make something up to sound smart.  
  This is called **Hallucination**.  

- **With RAG:**  
  Before you answer, I hand you a newspaper from today.  
  You read the specific article relevant to my question, and then you answer using that information.  

**RAG is simply the process of handing the LLM the right notes before it speaks.**

#### 2. Visual-in-Words: The Architecture  
Close your eyes and visualize this flow:  

1. The **User** asks a question:  
   *"How do I reset the X-2000 router?"*  

2. The **Retriever** (The Hand) reaches into a bucket of manuals (your database), rummages around, and grabs the one specific paragraph about resetting the X-2000.  

3. The **Augmenter** (The Glue) takes the User's question and tapes that paragraph to it.  
   New Prompt:  
   *"Context: [Reset paragraph]. Question: How do I reset the X-2000 router?"*  

4. The **Generator** (The Brain / ollama) reads the combined text and generates an accurate answer.

In [1]:
import ollama
import chromadb
from chromadb.utils import embedding_functions

# 1. Define the question
user_question = "What is the capital of India?"

# 2. Send it to the model (Generation only)
# We are NOT providing external context yet.
response = ollama.chat(
    model='gemma3:4b',  # Enter the model that you pull on ollama
    messages=[{'role': 'user', 'content': user_question}]
)

# 3. Print the answer
print("AI Answer:", response['message']['content'])

AI Answer: The capital of India is **New Delhi**. 

It's a planned city, officially named New Delhi, though the larger metropolitan area is often referred to as Delhi. 😊 

Do you want to know anything more about New Delhi or India in general?


We write,<br>
`response['message']['content']`<br>
because it is in JSON format:<br>

```json
{
  "model": "phi3:3.8b",
  "created_at": "2025-12-15T09:45:00.5105742Z",
  "done": true,
  "done_reason": "stop",
  "total_duration": 11129766800,
  "load_duration": 3728855400,
  "prompt_eval_count": 16,
  "prompt_eval_duration": 765201100,
  "eval_count": 94,
  "eval_duration": 6592721000,
  "message": {
    "role": "assistant",
    "content": "The capital of India is New Delhi. It became the political center in 1956 when the country's government moved from Kolkata (then called Calcutta) and now serves as a major hub for culture, education, and politics, housing Parliament House, which holds the Indian legislative bodies - Lok Sabha (House of the People), Rajya Sabha (Council of States), along with numerous other government offices.",
    "thinking": null,
    "images": null,
    "tool_name": null,
    "tool_calls": null
  },
  "logprobs": null
}



### Different Roles

| Role      | Analogy               | Purpose                                  |
|-----------|-----------------------|------------------------------------------|
| System    | The Boss / Director   | Defines behavior, tone, and rules.        |
| User      | The Customer          | Asks the question.                       |
| Assistant | The Transcript        | Provides conversation history/context.   |


```py
messages=[
    # 1. We set the context (optional but good)
    {'role': 'system', 'content': 'You are a helpful assistant.'},

    # 2. First turn
    {'role': 'user', 'content': 'Who wrote Harry Potter?'},
    {'role': 'assistant', 'content': 'J.K. Rowling wrote it.'}, # <--- WE INSERT THIS MANUALLY OR FROM PREVIOUS RESPONSE

    # 3. Current question (The AI now reads the line above and knows 'she' = Rowling)
    {'role': 'user', 'content': 'What year was she born?'}
]

## Embedding

#### 1. Intuition: The Supermarket Aisle  
Imagine you go to a supermarket and ask a clerk: “I need ingredients for a salad.”  

- **Keyword Search (The old Way):**  
The clerk looks for items that have the word “Salad” written on them.  
They hand you “Salad Dressing” and “Salad Tongs”.  
They miss the Lettuce and Tomatoes because those items don’t have the word “Salad” printed on them.  

- **Vector Search (Embeddings – The RAG Way):**  
The clerk understands the concept of a salad.  
They know that Lettuce, Tomatoes, and Cucumbers “live” in the same conceptual aisle as “Salad”, even if the words are different.  

Embeddings turn text into a coordinate on a map.  
Words with similar meanings live close together.

#### 2. Visual-in-Words: The 3-D Map  
Imagine a giant 3-D box floating in space.  
Every sentence you can possibly speak is a tiny dot inside this box.  

- The dot for “The dog barked” is at coordinate [10, 50, 3].  
- The dot for “The puppy made a noise” is at [11, 51, 4]. (Very close!)  
- The dot for “I like pizza” is at [90, 2, 88]. (Far away!)

The Translator (Embedding Model) is the tool that takes your text and gives you those coordinates.

In [2]:
# 1. The text we want to translate
text = "The cat watched carefully as the playful kitten chased a leaf across the driveway, only stopping when a car rolled slowly past the house."


# 2. Ask Ollama to create the embedding
# We use 'nomic-embed-text' because it's built for this.
response = ollama.embeddings(
    model='nomic-embed-text',
    prompt=text
)

# 3. Get the vector (The list of numbers)
vector = response['embedding']

# 4. Inspect the result
print(f"Text: {text}")
print(f"Vector Length: {len(vector)}") # How many dimensions?
print(f"First 5 numbers: {vector[:5]}") # Just a peek

Text: The cat watched carefully as the playful kitten chased a leaf across the driveway, only stopping when a car rolled slowly past the house.
Vector Length: 768
First 5 numbers: [1.0189101696014404, 0.19626553356647491, -2.8098843097686768, 0.09355314821004868, 1.6594434976577759]


## Vector Stores

#### 1. Intuition: The Color Gradient

Imagine a library where books aren't sorted by author or title, but by **color**.

- All the **"Red"** books (Action/Adventure) are in one corner.
- All the **"Blue"** books (Sad/Melancholy) are in another.

When you walk in and say *"I want something Red-ish,"* the librarian doesn't check every book in the building.  
They walk straight to the Red corner and grab the closest matches.

A **Vector Store** is that library.  
It organizes your data so that *"math-similar"* items sit next to each other in memory.

#### 2. Visual-in-Words: The Index

- **Without Vector Store:**  
  You have a pile of 1,000 messy papers on the floor.  
  To find *"Project X,"* you pick up paper #1, read it, put it down.  
  Pick up #2, read it… *(slow)*

- **With Vector Store:**  
  You have a specialized filing cabinet where related topics are magnetized together.  
  You magnetize your query (*"Project X"*) and throw it at the cabinet.  
  It sticks immediately to the right folder.



In [3]:

# 1. Initialize the Client (The Librarian)
# This creates a temporary in-memory database. 
# (Use persistent_client to save to disk, but for now, memory is fine)
client = chromadb.Client()

# 2. Create a Collection (The Bookshelf)
# A 'collection' is where we store related documents.
collection = client.create_collection(name="dragon_facts") #here since we didn't add any model, so it will download a default embedding model

# 3. Add Documents (The Knowledge)
# We give each chunk of text an ID so we can find it later.
documents = [
    "My dragon's name is Fluffy.",
    "Fluffy eats 500kg of spicy tacos every Tuesday.",
    "Fluffy lives in a volcano behind the Walmart."
]
ids = ["id1", "id2", "id3"]

# 4. Store them
# NOTICE: Chroma handles the embedding part automatically in the background!
collection.add(
    documents=documents,
    ids=ids
)

print("Library built! I currently know", collection.count(), "facts about the dragon.")

Library built! I currently know 3 facts about the dragon.


## Retrival

#### 1. Intuition: The "Needle in the Haystack"

You have a massive haystack (your database).  
You throw a magnet (your query) into it.  
The magnet pulls out the 3 needles that are most magnetic (most similar).

**Retrieval is simply:** Query Vector vs. Database Vectors.

#### 2. Visual-in-Words: The Search

1. **User Query:**  
   *"What does the dragon eat?"*

2. **Translation:**  
   We convert that question into a vector (numbers).

3. **Comparison:**  
   The database compares that vector against:
   - "Fluffy's Name"
   - "Fluffy's Food"
   - "Fluffy's Home"

4. **Ranking:**
   - Match with **"Fluffy Name"**: 10% match  
   - Match with **"Fluffy Home"**: 15% match  
   - Match with **"Fluffy Food"**: 92% match  

5. **Result:**  
   It returns the **"Food"** document.


In [4]:
# 1. Setup (Same as before)
client = chromadb.Client()
collection = client.create_collection(name="dragon_facts_test")

documents = [
    "My dragon's name is Fluffy.",
    "Fluffy eats 500kg of spicy tacos every Tuesday.",
    "Fluffy lives in a volcano behind the Walmart."
]
collection.add(documents=documents, ids=["id1", "id2", "id3"])

# ---------------------------------------------------------
# 2. The Retrieval Step
# ---------------------------------------------------------

query_text = "Tell me about the creature's diet"

results = collection.query(
    query_texts=[query_text],
    n_results=1  # How many top matches do you want? Let's ask for the best ONE.
)

# 3. Inspect the results
print("Question:", query_text)
print("\n--- Retrieved Document ---")
print(results['documents'][0][0]) # It returns a list of lists, so we grab the first item.

Question: Tell me about the creature's diet

--- Retrieved Document ---
Fluffy eats 500kg of spicy tacos every Tuesday.


## The Full RAG Pipeline

1. Setup: Initialize Ollama (Brain) and Chroma (Library).
2. Ingest: Read data, Embed it (using Ollama), Store it.
3. Ask: User asks a question.
4. Retrieve: Find the best chunk.
5. Augment: Create a prompt: "Context: {chunk}. Question: {question}."
6. Generate: Get the final answer.

In [5]:
# ---------------------------------------------------------
# 1. SETUP: Connect the Brain and the Library
# ---------------------------------------------------------

# We tell Chroma: "Don't use your default. Use Ollama's 'nomic-embed-text' model."
# Make sure you ran: `ollama pull nomic-embed-text` in terminal first!
ollama_ef = embedding_functions.OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings", # Standard Ollama endpoint
)

client = chromadb.Client()
collection = client.create_collection(
    name="dragon_rag", 
    embedding_function=ollama_ef # <--- KEY CHANGE: Using Ollama for embeddings
)

# ---------------------------------------------------------
# 2. INGEST: Teach the AI
# ---------------------------------------------------------
# Pro Tip: Repeating the subject ("The dragon Fluffy") helps retrieval!
docs = [
    "The dragon's name is Fluffy.",
    "The dragon Fluffy eats 500kg of spicy tacos every Tuesday.",
    "The dragon Fluffy lives in a volcano behind the Walmart."
]
ids = ["id1", "id2", "id3"]

collection.add(documents=docs, ids=ids)

# ---------------------------------------------------------
# 3. ASK & RETRIEVE
# ---------------------------------------------------------
user_question = "Where does the beast live?" # Tricky question (Beast != Dragon)

# Look it up!
results = collection.query(
    query_texts=[user_question],
    n_results=1
)
best_chunk = results['documents'][0][0]

print(f"🔎 Retrieved Context: '{best_chunk}'")

# ---------------------------------------------------------
# 4. AUGMENT & GENERATE
# ---------------------------------------------------------
# This is the "Prompt Engineering" part.
prompt = f"""
You are an expert on mythical creatures.
Use the following context to answer the question.
If the answer is not in the context, say "I don't know".

Context: {best_chunk}

Question: {user_question}
"""

print("\n🤖 Generating Answer...")
response = ollama.chat(
    model='gemma3:4b', # Or 'llama3' or 'mistral'
    messages=[{'role': 'user', 'content': prompt}]
)

print("--------------------------------------------------")
print(response['message']['content'])
print("--------------------------------------------------")

🔎 Retrieved Context: 'The dragon Fluffy lives in a volcano behind the Walmart.'

🤖 Generating Answer...
--------------------------------------------------
The beast lives in a volcano behind the Walmart.
--------------------------------------------------


## The Butcher (Chunking)

#### 1. Intuition: Eating a Steak  
You cannot swallow a whole steak in one go. You will choke.  
You must use a knife to cut it into bite-sized pieces.  

- The **Steak**: Your long document.  
- The **Knife**: The Chunker.  
- The **Bite**: The Chunk (Vector).

#### 2. Visual-in-Words: The Overlap  
Imagine cutting a photograph into strips.  
If you cut exactly on the edge of a face, you might lose who it is.  

So, we use **Overlapping Chunks**.  

Chunk 1: *"...The dragon lives in a volcano. It is hot..."*  
Chunk 2: *"...volcano. It is hot and filled with lava..."*

In [6]:
import ollama
import chromadb
from chromadb.utils import embedding_functions

# ---------------------------------------------------------
# 1. The "Long" Document
# ---------------------------------------------------------
text_file = """
COMPANY LEAVE POLICY 2024
1. Sick Leave: Employees are entitled to 10 days of sick leave per year. 
   To claim sick leave, a medical certificate is required for absences longer than 2 days.
2. Casual Leave: Employees get 5 days of casual leave. This must be applied for 24 hours in advance.
3. Remote Work: Remote work is allowed on Fridays only. 
   You must be logged into Slack by 9:00 AM.
4. Lunch Break: Lunch is from 1:00 PM to 2:00 PM. 
   Eating at desks is strictly prohibited to keep equipment clean.
5. Termination: The company requires 2 weeks of notice before resignation.
"""

# ---------------------------------------------------------
# 2. The Butcher Function (Chunking)
# ---------------------------------------------------------
def chunk_text(text, chunk_size=100, overlap=20):
    # This is a very simple character-based splitter
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        # Create the chunk
        chunk = text[start:end]
        chunks.append(chunk)
        # Move the window forward, but step back by 'overlap' amount
        start += (chunk_size - overlap) 
    return chunks

# Chop the meat!
my_chunks = chunk_text(text_file, chunk_size=150, overlap=30)

print(f"Original Text Length: {len(text_file)}")
print(f"Number of Chunks: {len(my_chunks)}")
print("--- Sample Chunk 1 ---")
print(f"'{my_chunks[1]}'") # Printing the second chunk to see it

# ---------------------------------------------------------
# 3. Setup Database (The same as Lesson 5)
# ---------------------------------------------------------
ollama_ef = embedding_functions.OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings"
)

client = chromadb.Client()
# Important: We name it differently so we start fresh
collection = client.create_collection(name="policy_rag", embedding_function=ollama_ef)

# ---------------------------------------------------------
# 4. Ingest Chunks
# ---------------------------------------------------------
# Create IDs automatically: "id_0", "id_1", "id_2"...
ids = [f"id_{i}" for i in range(len(my_chunks))]

collection.add(documents=my_chunks, ids=ids)

# ---------------------------------------------------------
# 5. Ask a Question
# ---------------------------------------------------------
question = "Can I work from home?"

results = collection.query(query_texts=[question], n_results=1)
best_chunk = results['documents'][0][0]

print(f"\n🔎 Found Policy: '{best_chunk}'")

# ---------------------------------------------------------
# 6. Generate Answer
# ---------------------------------------------------------
prompt = f"Context: {best_chunk}\nQuestion: {question}\nAnswer:"

response = ollama.chat(model='gemma3:4b', messages=[{'role': 'user', 'content': prompt}])
print("\n🤖 AI Answer:", response['message']['content'])

Original Text Length: 588
Number of Chunks: 5
--- Sample Chunk 1 ---
'ave, a medical certificate is required for absences longer than 2 days.
2. Casual Leave: Employees get 5 days of casual leave. This must be applied fo'

🔎 Found Policy: 'leave. This must be applied for 24 hours in advance.
3. Remote Work: Remote work is allowed on Fridays only. 
   You must be logged into Slack by 9:00'

🤖 AI Answer: Yes, you can work from home, but only on Fridays and you must be logged into Slack by 9:00 AM. Also, you need to apply for leave 24 hours in advance.


## The Elephant's Brain (Chat History)  
#### 1. Intuition: The Amnesia Patient & The Diary  
The Problem:  
If you ask: *"Who is Fluffy?"* → It answers: *"A dragon."*  
Then you ask: *"What does he eat?"* → It asks: *"Who is 'he'?"*  

The Solution (Memory):  
Since the AI has amnesia, we must keep a **Diary** (List) of everything said so far.  
Every time we send a new message, we hand the entire Diary to the AI so it can catch up on the context before answering.

#### 2. Visual-in-Words: The Stack of Plates  
Turn 1: You put a plate down (User: *"Hi"*).  
The AI puts a plate on top (AI: *"Hello"*).  

Turn 2: You put a new plate (User: *"How are you?"*).  

The **Context Window**:  
The AI looks at the entire stack of plates from bottom to top to understand the conversation flow.

In [7]:
# ---------------------------------------------------------
# 1. SETUP (Same as before)
# ---------------------------------------------------------
ollama_ef = embedding_functions.OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings"
)
client = chromadb.Client()
collection = client.create_collection(name="dragon_chat_history2", embedding_function=ollama_ef)

# Add Data
docs = [
    "The dragon Fluffy eats 500kg of spicy tacos.",
    "Fluffy lives in a volcano."
]
ids = ["id1", "id2"]
collection.add(documents=docs, ids=ids)

# ---------------------------------------------------------
# 2. INITIALIZE MEMORY
# ---------------------------------------------------------
# We start with a System message to set the behavior
conversation_history = [
    {'role': 'system', 'content': 'You are a helpful assistant. Be brief.'}
]

print("💬 Chat initialized! (Type 'exit' to stop)")

# ---------------------------------------------------------
# 3. THE CHAT LOOP
# ---------------------------------------------------------
while True:
    # A. Get User Input
    user_input = input("\nYou: ")
    if user_input.lower() == "exit":
        break

    # B. Retrieval (RAG)
    # We look for facts related to the CURRENT question
    results = collection.query(query_texts=[user_input], n_results=1)
    context_data = results['documents'][0][0]

    # C. Construct the Prompt with Context
    # We don't save the HUGE context to history (it wastes space), 
    # we just use it for this specific turn.
    prompt_with_context = f"""
    
    You are a strict assistant. You must ONLY use the provided Context info to answer.
    If the answer is not in the Context info, you must say "I don't know".
    Do not use your own external knowledge.

    Context info: {context_data}
    User Question: {user_input}
    """

    # D. Prepare Message List for the AI
    # Take the FULL history + the NEW augmented prompt
    messages_to_send = conversation_history + [{'role': 'user', 'content': prompt_with_context}]

    # E. Generate
    response = ollama.chat(
        model='gemma3:4b',
        messages=messages_to_send
    )
    ai_answer = response['message']['content']
    print(f"AI: {ai_answer}")

    # F. Update Memory (CRITICAL STEP)
    # We save the Original User Question (without the messy context) and the AI Answer
    # This keeps the history clean for the next turn.
    conversation_history.append({'role': 'user', 'content': user_input})
    conversation_history.append({'role': 'assistant', 'content': ai_answer})

💬 Chat initialized! (Type 'exit' to stop)
AI: Fluffy.
AI: 500kg of spicy tacos.
AI: In a volcano.
AI: I don't know.


In [10]:
conversation_history

[{'role': 'system', 'content': 'You are a helpful assistant. Be brief.'},
 {'role': 'user', 'content': 'Who is the dragon?'},
 {'role': 'assistant', 'content': 'Fluffy.'},
 {'role': 'user', 'content': 'What does he eat?'},
 {'role': 'assistant', 'content': '500kg of spicy tacos.'},
 {'role': 'user', 'content': 'Where does he live?'},
 {'role': 'assistant', 'content': 'In a volcano.'},
 {'role': 'user', 'content': 'Where is the volcano?'},
 {'role': 'assistant', 'content': "I don't know."}]

## The Judge (Evaluation)

You have built the car. You have driven it. You have seen it crash (hallucinate).  
Now, how do we measure how good it is without manually chatting for hours?

#### 1. Intuition: The Exam  
- **Manual Testing:** You ask 100 questions. You read 100 answers. (Takes all day).  
- **Automated Evaluation:** You have an “Answer Key”. You write a script to check if the AI’s answer matches the key.

#### 2. Visual-in-Words: RAGAS (The Scorecard)  
There is a standard framework called **RAGAS** (Retrieval-Augmented Generation Assessment). It measures:  

- **Faithfulness:** Did the AI stick to the context? (No “Mystaria”.)  
- **Answer Relevance:** Did it actually answer the question? (No “I like turtles.”)  
- **Context Precision:** Did the retrieval find the right chunk?  

Since installing RAGAS libraries can be heavy, we will build a **Simple Judge** using the LLM itself.

In [8]:
# ---------------------------------------------------------
# The Judge Function
# ---------------------------------------------------------
def evaluate_rag(question, context, answer):
    # The Rubric for the Judge
    judge_prompt = f"""
    You are a grader. 
    1. Read the QUESTION.
    2. Read the CONTEXT (The facts).
    3. Read the ANSWER (What the AI said).
    
    Task: Determine if the ANSWER is fully supported by the CONTEXT.
    If the answer contains information NOT in the context (hallucination), score it 0.
    If the answer is faithful to the context, score it 1.
    
    Return ONLY the number: 0 or 1.
    
    QUESTION: {question}
    CONTEXT: {context}
    ANSWER: {answer}
    """
    
    response = ollama.chat(
        model='llama3.1:8b-instruct-q4_K_M', # The Judge needs to be smart!
        messages=[{'role': 'user', 'content': judge_prompt}]
    )
    
    score = response['message']['content'].strip()
    return score

# ---------------------------------------------------------
# Test Case 1: The Good Answer
# ---------------------------------------------------------
q1 = "Where does he live?"
c1 = "Fluffy lives in a volcano behind Walmart."
a1 = "He lives in a volcano behind Walmart." # Good answer

score1 = evaluate_rag(q1, c1, a1)
print(f"Test 1 Score (Expected 1): {score1}")

# ---------------------------------------------------------
# Test Case 2: The Hallucination (Mystaria)
# ---------------------------------------------------------
q2 = "Where is the volcano?"
c2 = "Fluffy lives in a volcano behind Walmart."
a2 = "The volcano is on the island of Mystaria." # BAD answer

score2 = evaluate_rag(q2, c2, a2)
print(f"Test 2 Score (Expected 0): {score2}")

q3 = "What is 2+2?"
c3 = "The dragon eats tacos."
a3 = "4"

score3 = evaluate_rag(q3,c3,a3)
print(f"Test 3 score (Expected 0): {score3}")

Test 1 Score (Expected 1): 1
Test 2 Score (Expected 0): 0
Test 3 score (Expected 0): 1

The answer "4" is a simple arithmetic calculation that is not related to the context about dragons eating tacos. It accurately represents the sum of 2 + 2, making it faithful to the math context implied by the question itself, but not relevant or derived from the provided context. Therefore, I score it 1 for being an accurate answer in isolation.


Score3 is a **FAILURE** of the Judge!  
**Why?**

Context: *"The dragon eats tacos."*  
Answer: *"4"*.  

Does the sentence *"The dragon eats tacos"* support the fact that 2+2=4?  
**NO.**  
Therefore, the score should be **0**.

The Judge (Llama 3.2) got confused because it *knows* 2+2=4 is true in the real world, so it hesitated to mark it “False”.  
It didn’t strictly follow the rule **“Supported by Context”**.

How to make the Judge stricter?  
You need to explicitly tell the Judge to **ignore its own brain**.

Change the prompt to:

```python
Task: Determine if the ANSWER can be derived EXCLUSIVELY from the CONTEXT.
    Ignore your own knowledge. 
    If the context does not contain the answer, score it 0.

In [9]:
# ---------------------------------------------------------
# The Judge Function
# ---------------------------------------------------------
def evaluate_rag(question, context, answer):
    # The Rubric for the Judge
    judge_prompt = f"""
    You are a grader. 
    1. Read the QUESTION.
    2. Read the CONTEXT (The facts).
    3. Read the ANSWER (What the AI said).
    
    Task: Determine if the ANSWER can be derived EXCLUSIVELY from the CONTEXT.
    Ignore your own knowledge. 
    If the context does not contain the answer, score it 0. Else 1.
    
    Return ONLY the number: 0 or 1.
    
    QUESTION: {question}
    CONTEXT: {context}
    ANSWER: {answer}
    """
    
    response = ollama.chat(
        model='llama3.1:8b-instruct-q4_K_M', # The Judge needs to be smart!
        messages=[{'role': 'user', 'content': judge_prompt}]
    )
    
    score = response['message']['content'].strip()
    return score

# ---------------------------------------------------------
# Test Case 1: The Good Answer
# ---------------------------------------------------------
q1 = "Where does he live?"
c1 = "Fluffy lives in a volcano behind Walmart."
a1 = "He lives in a volcano behind Walmart." # Good answer

score1 = evaluate_rag(q1, c1, a1)
print(f"Test 1 Score (Expected 1): {score1}")

# ---------------------------------------------------------
# Test Case 2: The Hallucination (Mystaria)
# ---------------------------------------------------------
q2 = "Where is the volcano?"
c2 = "Fluffy lives in a volcano behind Walmart."
a2 = "The volcano is on the island of Mystaria." # BAD answer

score2 = evaluate_rag(q2, c2, a2)
print(f"Test 2 Score (Expected 0): {score2}")

q3 = "What is 2+2?"
c3 = "The dragon eats tacos."
a3 = "4"

score3 = evaluate_rag(q3,c3,a3)
print(f"Test 3 score (Expected 0): {score3}")

Test 1 Score (Expected 1): 1
Test 2 Score (Expected 0): 0
Test 3 score (Expected 0): 0


## Final Graduation Project
Objective: Build a “Personal Journal Chatbot”.

Data:  
Create a text file `journal.txt`. Write fake entries about your life.  
Example lines:  
- Jan 1: I started learning Guitar.  
- Jan 5: My guitar string broke.  
- Feb 10: I bought a new blue guitar.

Pipeline:
1. Load and chunk the file.  
2. Store in ChromaDB (using Ollama embeddings).  
3. Create a loop (`while True:`).  
4. Add memory (history).  
5. Use the strict prompt: *“I don’t know if not in context.”*  

Test:
- Ask: *“What instrument do I play?”*  
- Ask: *“What happened in January?”*  
- Ask: *“What color is my car?”* (Should reply *“I don’t know.”*)

#### Better Test Cases
1. The “Specific Detail” Test  
Q: *“What did I buy in February?”*  
Expected: Mention the **blue guitar** (Feb 10).  
Must **not** mention the capo (Jan 22).

2. The “Synthesis” Test (Combining facts)  
Q: *“Did I ever perform in public?”*  
Expected: Find **Mar 9 (Open Mic)** or **Jul 20 (Coffee Shop)** and reply **“Yes”**.

3. The “Inference” Test (Logic)  
Q: *“Do I know how to read music?”*  
Expected: Spot **Oct 2 (“I learned to read simple tablature”)** and say **“Yes”** or **“You can read tablature”**.

4. The “Trick” Question (The Pivot)  
Q: *“Do I play the piano?”*  
Expected: **“I don’t know”** (or **“No”**).  
Reasoning: **May 21** says you *collaborated with a pianist*—you are **not** the pianist.

5. The “Goal” Question  
Q: *“What do you want to do next year?”*  
Expected: Find **Dec 5** and mention **“Music Theory”**.

In [ ]:
with open ("journal.txt", "r", encoding="utf-8") as file:
    text_file = file.read()

def text_chunk(text, chunk_size=None, overlap=None):
    # 1. We ignore chunk_size/overlap because we are trusting the document structure.
    # 2. Split the text wherever there is a double newline.
    chunks = text.split('\n\n')
    
    # 3. Clean up: Remove any empty items or extra whitespace
    clean_chunks = [c.strip() for c in chunks if c.strip()]
    
    return clean_chunks

my_chunks = text_chunk(text_file, chunk_size=250, overlap= 75)

ollama_ef = embedding_functions.OllamaEmbeddingFunction(model_name= "nomic-embed-text", url="http://localhost:11434/api/embeddings")
client = chromadb.Client()
collection = client.create_collection(name="my_collection", embedding_function= ollama_ef)

ids = [f"id_{i}" for i in range(len(my_chunks))]

collection.add(documents= my_chunks, ids=ids)

convo_history = [
    {'role':'system', 'content':'You are a helpful assitant. Be brief.'}
]
print("Chat initialized! (Type 'exit' to stop)")

while True:

    user_input = input("/nYou")
    if user_input.lower() == 'exit':
        break

    result = collection.query(query_texts=[user_input], n_results=5)
    list_of_chunks = result['documents'][0]
    context_data = "\n\n".join(list_of_chunks)

    print(f"\n--- 🔎 DEBUG: Retrieved {len(list_of_chunks)} chunks ---")
    print(context_data)
    print("--------------------------------------------------\n")

    prompt_with_context = f"""
    You are a strict assistant. You must ONLY use the provided Context info to answer.
    If the answer is not in the Context info, you must say "I don't know".
    Do not use your own external knowledge.

    Context info: {context_data}
    User Question: {user_input}
    """

    messages_to_send = convo_history + [{'role': 'user', 'content': prompt_with_context}]    

    response = ollama.chat(
        model="llama3.1:8b-instruct-q4_K_M",
        messages=messages_to_send
    )
    ai_answer = response['message']['content']
    print(f"AI: {ai_answer}")

    convo_history.append({'role': 'user', 'content': user_input})
    convo_history.append({'role': 'assistant', 'content': ai_answer})

Chat initialized! (Type 'exit' to stop)

--- 🔎 DEBUG: Retrieved 5 chunks ---
My Journal

Jan 1: I started learning Guitar.

Jan 8: I practiced chords for an hour and finally switched between G and C without fumbling.

Jan 22: I bought a set of extra guitar strings and a capo.

Feb 10: I bought a new blue guitar.
--------------------------------------------------

AI: A new blue guitar.

--- 🔎 DEBUG: Retrieved 5 chunks ---
My Journal

Mar 9: I discovered a local open mic and signed up to play a 2-minute song.

Jul 20: I played a short set at the neighborhood coffee shop; it felt exhilarating.

Jan 18: I met a neighbor who used to play in a college band and gave me tips.

May 21: I collaborated with a pianist online for a cover song.
--------------------------------------------------

AI: Yes.

--- 🔎 DEBUG: Retrieved 5 chunks ---
Oct 2: I learned to read simple tablature and it unlocked lots of new songs.

Mar 3: I learned about tuning by ear and practiced matching notes.

Feb 1: I learn

In [2]:
convo_history

[{'role': 'system', 'content': 'You are a helpful assitant. Be brief.'},
 {'role': 'user', 'content': 'What did I buy in February?'},
 {'role': 'assistant', 'content': 'A new blue guitar.'},
 {'role': 'user', 'content': 'Did I ever perform in public?'},
 {'role': 'assistant', 'content': 'Yes.'},
 {'role': 'user', 'content': 'Do I know how to read music?'},
 {'role': 'assistant', 'content': 'No.'},
 {'role': 'user', 'content': 'Do I play the piano?'},
 {'role': 'assistant', 'content': 'No.'},
 {'role': 'user', 'content': 'What do you want to do next year?'},
 {'role': 'assistant', 'content': 'I want to learn music theory.'}]

In [ ]:
# Code for Universal chunker incase, we dont know the document structure.
import re

def universal_chunker(text, target_chunk_size=500, overlap_size=100):
    # 1. Clean the text (remove excessive newlines)
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 2. Split into sentences using Regex (looks for punctuation followed by space)
    # This splits by ".", "!", or "?"
    sentences = re.split(r'(?<=[.!?]) +', text)
    
    chunks = []
    current_chunk = []
    current_length = 0
    
    for sentence in sentences:
        sentence_len = len(sentence)
        
        # Check if adding this sentence would exceed the limit
        if current_length + sentence_len > target_chunk_size:
            # A. Save the current chunk
            chunks.append(" ".join(current_chunk))
            
            # B. Handle Overlap (The Safety Net)
            # We keep the last few sentences so the context flows into the next chunk
            overlap_buffer = []
            overlap_count = 0
            
            # Go backwards through the current chunk to grab sentences for overlap
            for s in reversed(current_chunk):
                if overlap_count < overlap_size:
                    overlap_buffer.insert(0, s) # Add to front
                    overlap_count += len(s)
                else:
                    break # Stop when we have enough overlap
            
            # Start the new chunk with the overlap sentences
            current_chunk = overlap_buffer
            current_length = overlap_count
            
        # Add the new sentence
        current_chunk.append(sentence)
        current_length += sentence_len
        
    # Add the final leftover bits
    if current_chunk:
        chunks.append(" ".join(current_chunk))
        
    return chunks